In [1]:
import pandas as pd
datafile = "titanic.csv"
data = pd.read_csv(datafile)

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
data['Gender']=data['Sex'].apply(lambda a: 1 if a=='male' else 0)

data=data.drop(['PassengerId','Name','Cabin','Embarked','Ticket','Sex'],axis=1)

data=data.dropna()

data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1


In [3]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size = 0.2)

In [4]:
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
879,1,1,56.0,0,1,83.1583,0
586,0,2,47.0,0,0,15.0000,1
753,0,3,23.0,0,0,7.8958,1
721,0,3,17.0,1,0,7.0542,1
798,0,3,30.0,0,0,7.2292,1


In [5]:
test.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
439,0,2,31.0,0,0,10.5000,1
619,0,2,26.0,0,0,10.5000,1
433,0,3,17.0,0,0,7.1250,1
197,0,3,42.0,0,1,8.4042,1
458,1,2,50.0,0,0,10.5000,0


In [6]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(max_leaf_nodes=15)
#clf=DecisionTreeClassifier()
clf=clf.fit(train.drop(['Survived'],axis=1).values,train["Survived"].values)

In [7]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=15, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [8]:
clf.feature_importances_

array([ 0.21420551,  0.17261662,  0.06142873,  0.        ,  0.0844932 ,
        0.46725594])

In [9]:
from sklearn import tree
with open("titanic.dot","w") as f:
    f = tree.export_graphviz(clf,
                            feature_names=train.drop(['Survived'],axis=1).columns,out_file=f)

In [10]:
predictions = clf.predict(test.drop(['Survived'],axis=1).values)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(test["Survived"], predictions)

0.78321678321678323

In [12]:
from sklearn.ensemble import RandomForestClassifier

X_train=train.drop(['Survived'],axis=1).values
y_train=train["Survived"].values
X_test=test.drop(['Survived'],axis=1).values
y_test=test["Survived"]

clf = RandomForestClassifier(n_estimators=1000,max_leaf_nodes=15)
clf=clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.79020979020979021

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf=clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.79720279720279719

In [14]:
from sklearn import svm
clf = svm.SVC()

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.66433566433566438

In [15]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2")

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.68531468531468531

In [16]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.78321678321678323

In [17]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)                             

0.79020979020979021

In [18]:
## STACKING

clf = tree.DecisionTreeClassifier(max_leaf_nodes=25)
clf.fit(X_train,y_train)
prediction=clf.predict(X_train)

clf1 = GradientBoostingClassifier(n_estimators=100, learning_rate=.1,max_depth=1, random_state=0)
clf1.fit(X_train,y_train)
prediction1=clf1.predict(X_train)

clf2 = RandomForestClassifier(n_estimators=100)
clf2.fit(X_train,y_train)
prediction2=clf2.predict(X_train)

dataL2=pd.DataFrame(prediction,columns=['p1'])

dataL2=pd.concat([dataL2 ,pd.DataFrame(prediction1,columns=['p2']), pd.DataFrame(prediction2,columns=['p3'])],axis=1)

dataL2.head()

,p1,p2,p3
0,1,1,1
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [19]:
from sklearn.linear_model import LogisticRegression

L2X=dataL2.values
L2y=y_train

finalClassifier=LogisticRegression()
finalClassifier.fit(L2X,L2y)

# Test Data Generation
testPrediction=clf.predict(X_test)
testPrediction1=clf1.predict(X_test)
testPrediction2=clf2.predict(X_test)

testL2=pd.DataFrame(testPrediction,columns=['p1'])

testL2=pd.concat([testL2 ,pd.DataFrame(testPrediction1,columns=['p2']), pd.DataFrame(testPrediction2,columns=['p3'])],axis=1)

finalPred=finalClassifier.predict(testL2.values)

accuracy_score(y_test, finalPred) 

0.72727272727272729

In [20]:
import sklearn
sklearn.__version__

'0.18.1'